In [34]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [35]:
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

In [64]:
mesh_stern = 'mobley_test/1112_tetrachloroethane/surf_d02_stern'
mesh_diel = 'mobley_test/1112_tetrachloroethane/surf_d02'
atom_data = numpy.loadtxt('mobley_test/1112_tetrachloroethane/radii.xyzr')

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.#80.

atom_pos = atom_data[:,:3]
atom_r   = atom_data[:,3]
N_atom = len(atom_r)

eps_well = 0.3598/4.184 # kcal/mol
r_vdw = 1.9080

rho_w = 0.0336 # 1/angs3 number density of water at standard conditions

grid = generate_grid(mesh_diel)

N_panel = grid.leaf_view.entity_count(0)
vertices = grid.leaf_view.vertices 
triangles = grid.leaf_view.elements
elements = list(grid.leaf_view.entity_iterator(0))

area = numpy.zeros(N_panel)
center = numpy.zeros((N_panel,3))
normal = numpy.zeros((N_panel,3))
for i in range(N_panel):
    area[i] = elements[i].geometry.volume
    center[i,:] = numpy.average(elements[i].geometry.corners[:],axis=1)
    v1 = elements[i].geometry.corners[:,1] - elements[i].geometry.corners[:,0]
    v2 = elements[i].geometry.corners[:,2] - elements[i].geometry.corners[:,0]
    normal[i,:] = numpy.cross(v1,v2)/(2*area[i]) 

integral_i = numpy.zeros(N_atom)
for i in range(N_atom):
    r_local = center - atom_pos[i,:]
    r_norm = numpy.sqrt(numpy.sum(r_local**2, axis=1))
    rdn = numpy.sum(r_local*normal, axis=1)
    
    A = 4*eps_well*atom_r[i]**12
    B = 4*eps_well*atom_r[i]**6
    
 #   test= numpy.where(A/(90*r_norm**12)>1e10)[0]
 #   if len(test)>0:
 #       print test
 #       print r_norm[test]
 #       print elements[test[0]].geometry.corners[:]
 #       print center[test[0]]
 #       print atom_pos[i,:]
 #       print numpy.sum((elements[test[0]].geometry.corners[:,2] - atom_pos[i,:])**2)
        
    r_large = numpy.where(r_norm>0.4*atom_r[i])[0]
    print 'Triangles out: %i'%(len(center)-len(r_large))
    
    '''
    r_small = numpy.where(r_norm<0.2*atom_r[i])[0]
    if len(r_small)>0:
        print elements[r_small[0]].geometry.corners[:]
        print center[r_small[0]]
        print 'atom pos and radius'
        print atom_pos[i,:], atom_r[i]
        print 'distance to corners'
        print numpy.sqrt(numpy.sum((elements[r_small[0]].geometry.corners[:,0] - atom_pos[i,:])**2))
        print numpy.sqrt(numpy.sum((elements[r_small[0]].geometry.corners[:,1] - atom_pos[i,:])**2))
        print numpy.sqrt(numpy.sum((elements[r_small[0]].geometry.corners[:,2] - atom_pos[i,:])**2))
        print 'distance to center'
        print r_norm[r_small[0]]
    '''
    integral_i[i] = numpy.sum((-A/(90*r_norm[r_large]**12) + B/(12*r_norm[r_large]**6)) * rdn[r_large] * area[r_large])
    
energy = rho_w*numpy.sum(integral_i)
print energy
print integral_i

Triangles out: 0
Triangles out: 0
Triangles out: 0
Triangles out: 0
Triangles out: 0
Triangles out: 0
Triangles out: 0
Triangles out: 0
0.297691727837
[ 0.94180935  0.66131302  1.71526372  1.57004873  1.61004108  1.57426482
  0.39360306  0.39352909]


In [39]:
(1.244+3.177+0.105)/3

1.5086666666666668

In [ ]:
1.73310520523/0.14104208421

In [ ]:
print elements[0].geometry